In [1]:
def getdetail(url , title, category, cur):
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    dic = {"summary":soup.select('#summary')[0].text,
           "title": title,
           "category": category
    }
    sql = "insert into news_main({}) values({})"
    sql2 = sql.format(','.join(dic.keys()), ','.join(len(dic) * '?'))
    cur.execute(sql2, dic.values())
    

In [6]:
import requests
import sqlite3 
con = sqlite3.connect('news.sqlite')
cur = con.cursor()
cur.execute("""drop table news_main;""")
cur.execute("""create table news_main(summary text, title text, category varchar(20));""")
con.commit()
from bs4 import BeautifulSoup
domain = 'http://www.appledaily.com.tw'
for page in range(1,30):
    res = requests.get('http://www.appledaily.com.tw/realtimenews/section/new/' + str(page))
    soup = BeautifulSoup(res.text)
    for rtddt in soup.select('.rtddt'):
        title = rtddt.select('h1')[0].text
        category = rtddt.select('h2')[0].text
        if domain not in rtddt.select('a')[0]['href'] :
            url = domain + rtddt.select('a')[0]['href'] 
        else:
            url = rtddt.select('a')[0]['href'] 
        getdetail(url , title, category, cur)
        con.commit()
con.close()

In [16]:
import jieba
jieba.add_word('周子瑜')
for ele in jieba.cut('韓國女團「TWICE」成員、16歲台灣正妹周子瑜，回台參加學力鑑定考試並與親友師長會面後，結束短短不到2天行程，今清晨5時35分在隨行人員陪同下抵達高雄小港機場，5時39分進入出境管制門，將搭乘上午7時華航CI0164班機直飛韓國仁川機場，預計上午10時45分抵達'):
    print ele,

韓國 女團 「 TWICE 」 成員 、 16 歲 台灣 正妹 周子瑜 ， 回台 參加 學力 鑑定 考試並 與 親友 師長 會面 後 ， 結束 短短 不到 2 天 行程 ， 今 清晨 5 時 35 分在 隨 行人 員 陪同 下 抵達 高雄 小港 機場 ， 5 時 39 分進 入出境 管制 門 ， 將 搭乘 上午 7 時華航 CI0164 班機 直飛 韓國 仁川 機場 ， 預計 上午 10 時 45 分 抵達


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

ary = ['【更新】柯P：洪智坤洩漏公文案還沒看到公文　今處理',
'留洪智坤 柯：殘障求職不易',
'人事處議處洪智坤　柯P：不清楚議處結果']
corpus = []
for title in ary:
    corpus.append('|'.join(jieba.cut(title)))
print corpus[0]  
print corpus[1]  
print corpus[2]  
vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(corpus)
word = vectorizer.get_feature_names() 
for w in word:
    print w,
print 
print X.toarray()

In [29]:
import sqlite3 
import jieba
con = sqlite3.connect('news.sqlite')
cur = con.cursor()
data = cur.execute("""select * from news_main;""")
corpus = []
tags = []
for rec in data:
    if (rec[2] == '娛樂'.decode('utf-8')) or \
        (rec[2] == '社會'.decode('utf-8')) or \
        (rec[2] == '財經'.decode('utf-8')):
        summary = rec[0]
        corpus.append(' '.join(jieba.cut(summary)))
        tags.append(rec[2]) 

con.close()

In [30]:
import jieba
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(corpus)
word = vectorizer.get_feature_names() 

In [31]:
from sklearn.cross_validation import train_test_split
train_data, test_data, train_tag, test_tag = train_test_split(X, tags, test_size=0.30, random_state=42)

In [32]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(train_data,train_tag)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [33]:
pred = clf.predict(test_data)

In [34]:
from sklearn.metrics import accuracy_score
print accuracy_score(test_tag, pred)

0.875


In [35]:
from sklearn.metrics import confusion_matrix 
a = confusion_matrix(test_tag, pred) 
for ele in clf.classes_:
    print ele,
print
print a

娛樂 社會 財經
[[36  1  5]
 [ 2 21  1]
 [ 0  1 13]]


In [36]:
index = 0
coef_features_c1_c2_c3 = []
import re
for feat, c1, c2, c3 in zip(vectorizer.get_feature_names(), \
                        clf.feature_count_[0], clf.feature_count_[1], clf.feature_count_[2]):
    coef_features_c1_c2_c3.append(tuple([clf.coef_[0][index], feat, c1, c2, c3]))
    index+=1
        
for i in sorted(coef_features_c1_c2_c3):
    if i[2] == max(i[2], i[3],i[4]) and i[2] >0 and  re.match(u"[\u4e00-\u9fa5]+",i[1]):
        print "娛樂", i[1], i[2], i[3], i[4]

娛樂 一下 1.0 0.0 0.0
娛樂 一人 1.0 0.0 0.0
娛樂 一份 1.0 0.0 0.0
娛樂 一切 1.0 1.0 0.0
娛樂 一到 1.0 0.0 0.0
娛樂 一半 1.0 1.0 1.0
娛樂 一同 1.0 0.0 0.0
娛樂 一場 1.0 0.0 0.0
娛樂 一定 1.0 1.0 0.0
娛樂 一對 1.0 0.0 0.0
娛樂 一早 1.0 0.0 0.0
娛樂 一模 1.0 0.0 0.0
娛樂 一段 1.0 0.0 0.0
娛樂 一輩子 1.0 0.0 0.0
娛樂 一輪 1.0 0.0 0.0
娛樂 一過 1.0 0.0 0.0
娛樂 一長 1.0 0.0 0.0
娛樂 一雙 1.0 0.0 0.0
娛樂 三個 1.0 0.0 0.0
娛樂 上傳 1.0 0.0 0.0
娛樂 上升 1.0 0.0 0.0
娛樂 上星期 1.0 0.0 0.0
娛樂 上演 1.0 0.0 0.0
娛樂 上獻唱 1.0 0.0 0.0
娛樂 上車 1.0 0.0 0.0
娛樂 下去 1.0 0.0 0.0
娛樂 下留 1.0 0.0 0.0
娛樂 不來 1.0 0.0 0.0
娛樂 不僅 1.0 1.0 0.0
娛樂 不受 1.0 0.0 0.0
娛樂 不吃 1.0 0.0 0.0
娛樂 不實 1.0 0.0 0.0
娛樂 不少圈 1.0 0.0 0.0
娛樂 不想 1.0 0.0 0.0
娛樂 不棄 1.0 0.0 0.0
娛樂 不營養 1.0 0.0 0.0
娛樂 不用 1.0 0.0 0.0
娛樂 不禁 1.0 0.0 0.0
娛樂 不科學 1.0 0.0 0.0
娛樂 不能 1.0 1.0 0.0
娛樂 不要 1.0 0.0 0.0
娛樂 不過步 1.0 0.0 0.0
娛樂 不遠 1.0 0.0 0.0
娛樂 不離 1.0 0.0 0.0
娛樂 世紀夢幻 1.0 0.0 0.0
娛樂 並參加 1.0 0.0 0.0
娛樂 並在 1.0 0.0 0.0
娛樂 並寫 1.0 0.0 0.0
娛樂 並將 1.0 1.0 0.0
娛樂 並沒有 1.0 1.0 0.0
娛樂 並非 1.0 1.0 1.0
娛樂 中學力 1.0 0.0 0.0
娛樂 中射籃 1.0 0.0 0.0
娛樂 中成 1.0 0.0 0.0
娛樂 中間 1.0 0.0 0

## Clustering

In [59]:
import sqlite3 
import jieba
con = sqlite3.connect('news.sqlite')
cur = con.cursor()
data = cur.execute("""select * from news_main;""")
corpus = []
tags = []
title = []
for rec in data:    
    summary = rec[0]
    corpus.append(' '.join(jieba.cut(summary)))
    tags.append(rec[2]) 
    title.append(rec[1]) 

con.close()

In [73]:

import scipy as sp
def tfidf(t, d, D):
    tf = float(d.count(t)) / sum(d.count(w) for w in set(d))
    idf = sp.log(float(len(D)) / (len([doc for doc in D if t in doc])))
    return tf * idf

a, abb, abc = ["a"], ["a", "b", "b"], ["a", "b", "c"]
D = [a, abb, abc]

print tfidf('b', abb, D)

0.270310072072


In [74]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X = vectorizer.fit_transform(corpus)
tfidf = transformer.fit_transform(X)
weight = tfidf.toarray()   

In [86]:
from sklearn import cluster
c = cluster.KMeans(n_clusters=15)
k_data = c.fit_predict(weight)

In [91]:
#print k_data
for idx, grp in enumerate(k_data):
    if grp == 13:
        print title[idx]

鄉民拍下坐旁邊的正妹想追　照片中人回文了(0)
傳謝長廷任駐日代表　林永樂：非常合適(0)
屏東有什麼好　大家只想到墾丁嗎？(0)
機場捷運墊片斷裂逾萬片　換新才3個月竟再...(0)
【法廣RFI】菲律賓向美軍開放五個軍事基...(26)
朱雪璋出這招　正心道館緩拆(736)
台電：核一廠人員疏失　停機4天損失至少2...(1867)
影片告訴你　買新成屋看這3件事　　(673)
台灣景氣如何？　主計長：不太好(493)
【央廣RTI】哈薩克國會選舉  聚焦權力...(12)
北韓人民吃不飽　金正恩卻越來越腫(2552)
外交部爆性騷醜聞　藍委：丟臉丟大了(2521)
【更新】八仙塵爆嚴重灼傷　18歲男重生後...(63858)
AV明日花全裸綁緞帶　周六炸台叫賣原味內...(4815)
山區撿戰車砲彈殼　女子用來恐嚇前夫　(1255)
競標制挨轟剝削農民　台糖董：市場就是這樣...(503)
【央廣RTI】喬帥五度封王  稱霸印地安...(342)
子瑜考後簽名桌　成校園展示品(6557)
當眾槍殺角頭老大　輔大休學生判無期定讞(11053)
楊哲宜褪下球衣換西裝掌金酒兵符(2390)
台股盤中～開高走低　失守8800點(492)
【買房問蘋果】租屋處公用電器　須詳列租約...(682)
在地草根味　台語賦詩銷建案　(552)
修法處罰拒訪者？　主計長：研修中尚未提案(702)
機場捷運問題不斷　交長：要廠商提安全保證(920)
研華為萊爾富、綠地超市打造智慧便利店(1507)
辣孕婦穿搭教學　透明蕾絲bra＋破褲(4501)
超多機車騎士都愛「這樣做」！真的很糟糕(10717)
機場捷運軌道誤佔據緊急煞車　是北捷136...(2310)
窮國的悲哀　總理兄弟也去德國當難民(3137)
我兒愛吸毒　媽媽忍痛報警抓(3223)
嚇！貨車底下掛「嬰孩」　網友：會出車禍(9356)
焦點股～子公司添柴火　神達開高走高(618)
Angelababy瘋宋仲基　「黃曉明能...(5546)
艾瓦瑞茲滿貫砲　金鶯補強火力升級(1827)
布局運動機能品牌　外資高喊儒鴻目標價(1095)
川普與克魯茲罵戰　金凱瑞譏：阿呆與阿瓜(1244)
替代役男注意　符合這幾點可在自家公司服役(5945)
柯P被爆「爛死了」　曾助他上位推手這麼回(52720)
台幣開盤升破32.5元　升值